# Basic CNN

In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.metrics import classification_report

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
y_train = pd.read_csv('Train/trainLbls.csv', header=None, names=['label'])['label']
y_validation = pd.read_csv('Validation/valLbls.csv', header=None, names=['label'])['label']
X_test = pd.read_csv('Test/testVectors.csv', header=None).transpose()

In [3]:
img_width, img_height = 256, 256
train_data_dir = "Train/TrainImages"
validation_data_dir = "Validation/ValidationImages"
test_data_dir = "Test/TestImages"
n_train_samples = len(y_train)
n_validation_samples = len(y_validation)
n_test_samples = X_test.shape[0]
n_labels = len(y_train.unique())
batch_size = 8
epochs = 50

In [4]:
# ImageDataGenerator generates batches of normalised image data i.e.
# a format that the images must be in to be read by the Keras
# model
train_batches = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30
).flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "categorical")

Found 5830 images belonging to 29 classes.


In [5]:
validation_batches = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30
).flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = "categorical")

Found 2298 images belonging to 29 classes.


In [6]:
model = Sequential([
    Conv2D(8, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    Flatten(),
    Dense(n_labels, activation='softmax')
])

In [7]:
model.compile(optimizers.Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 8)       224       
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                14967741  
Total params: 14,967,965
Trainable params: 14,967,965
Non-trainable params: 0
_________________________________________________________________


In [5]:
int(n_train_samples/batch_size)

728.75

In [ ]:
model.fit_generator(train_batches, 
                    validation_data=validation_batches,
                    epochs=5,
                    verbose=2
                   )

Epoch 1/5
